In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from scipy.stats import chi2
from scipy.stats import chi2_contingency
np.random.seed(10)

# Load the two dataframes

In [143]:
# Load datasets with correct date time format
price_demand = pd.read_csv('price_demand_data.csv', parse_dates=['SETTLEMENTDATE'], dayfirst=True)
weather = pd.read_csv('weather_data.csv', parse_dates=['Date'], dayfirst=True)

# Prepare data 
Cleaning Weather dataset

In [148]:
# DATA imputation: convert 'Calm' to 0 for wind speed columns
weather = weather.replace(to_replace='Calm', value=0)
weather['9am wind speed (km/h)'] = weather['9am wind speed (km/h)'].apply(pd.to_numeric, errors='coerce')
weather['3pm wind speed (km/h)'] = weather['3pm wind speed (km/h)'].apply(pd.to_numeric, errors='coerce')


In [149]:
# Fill NaN with mean value for numeric cells
weather = weather.fillna(round(weather.mean(),1))

C:\Users\lhong\AppData\Local\Temp\ipykernel_9768\1273634250.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  weather = weather.fillna(round(weather.mean(),1))
C:\Users\lhong\AppData\Local\Temp\ipykernel_9768\1273634250.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  weather = weather.fillna(round(weather.mean(),1))


In [150]:
weather.isna().sum()

weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               243 non-null    datetime64[ns]
 1   Minimum temperature (°C)           243 non-null    float64       
 2   Maximum temperature (°C)           243 non-null    float64       
 3   Rainfall (mm)                      243 non-null    float64       
 4   Evaporation (mm)                   243 non-null    float64       
 5   Sunshine (hours)                   243 non-null    float64       
 6   Direction of maximum wind gust     240 non-null    object        
 7   Speed of maximum wind gust (km/h)  243 non-null    float64       
 8   Time of maximum wind gust          240 non-null    object        
 9   9am Temperature (°C)               243 non-null    float64       
 10  9am relative humidity (%)          243

# Wraggling Price and Demand dataset

In [151]:
# Replace PRICECATEGORY values with Numeric values in new column PRICE_NUMERIC
price_demand['PRICECATEGORY'].replace( {'LOW' : 0, 'MEDIUM' : 1, 'HIGH' : 2, 'EXTREME': 3 }, inplace=True)

# Extract Date and Time seperately from SETTLEMENTDATE in date time format
price_demand['Date'] = pd.to_datetime(price_demand['SETTLEMENTDATE'].dt.date)
price_demand['Time'] = price_demand['SETTLEMENTDATE'].dt.time

# Set 'Day' every 48 time rows according to the definiteion from the data source company - AEMO
price_demand['Day'] = price_demand.index // 48

## Finding the maximum daily energy usage

In [152]:

demand = price_demand

max_demand = demand.groupby('Day')['TOTALDEMAND'].transform(max) == demand['TOTALDEMAND']
max_demand = demand[max_demand].reset_index()
max_demand['Max_TOTALDEMAND'] = max_demand['TOTALDEMAND']
max_demand 

,index,REGION,SETTLEMENTDATE,TOTALDEMAND,PRICECATEGORY,Date,Time,Day,Max_TOTALDEMAND
0,34,VIC1,2021-01-01 17:30:00,5019.64,0,2021-01-01,17:30:00,0,5019.64
1,81,VIC1,2021-01-02 17:00:00,4964.35,0,2021-01-02,17:00:00,1,4964.35
2,132,VIC1,2021-01-03 18:30:00,4503.31,0,2021-01-03,18:30:00,2,4503.31
3,180,VIC1,2021-01-04 18:30:00,4764.18,0,2021-01-04,18:30:00,3,4764.18
4,225,VIC1,2021-01-05 17:00:00,4800.64,0,2021-01-05,17:00:00,4,4800.64
...,...,...,...,...,...,...,...,...,...
238,11461,VIC1,2021-08-27 19:00:00,6769.89,2,2021-08-27,19:00:00,238,6769.89
239,11509,VIC1,2021-08-28 19:00:00,5716.32,1,2021-08-28,19:00:00,239,5716.32
240,11557,VIC1,2021-08-29 19:00:00,6227.89,3,2021-08-29,19:00:00,240,6227.89
241,11604,VIC1,2021-08-30 18:30:00,6072.91,1,2021-08-30,18:30:00,241,6072.91


## Finding the maximum daily price category

In [153]:
price = price_demand

max_price = price.groupby('Day')['PRICECATEGORY'].max()
max_price = max_price.reset_index()
max_price['Max_PRICECATEGORY'] = max_price['PRICECATEGORY']
max_price = max_price.drop(columns = ['PRICECATEGORY'])
max_price

,Day,Max_PRICECATEGORY
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
238,238,3
239,239,1
240,240,3
241,241,2


In [154]:
weather.set_index('Date')
max_demand.set_index('Date')
dataset = max_demand.merge(max_price)
dataset = dataset.merge(weather)

dataset = dataset.drop(columns = ['index','REGION','SETTLEMENTDATE'])

In [155]:
dataset

dataset.to_csv('all_data.csv',index=False)

In [156]:
# Pairwise pearson r correlation matrix between different variables
corr = dataset[['Minimum temperature (°C)', 'Maximum temperature (°C)', 'Rainfall (mm)', \
                'Evaporation (mm)', 'Sunshine (hours)', 'Speed of maximum wind gust (km/h)', \
                '9am Temperature (°C)', '3pm Temperature (°C)' ]].corr(method='pearson')

# corr = dataset[['Minimum temperature (°C)', 'Maximum temperature (°C)', 'Rainfall (mm)', \
#                 'Evaporation (mm)', 'Sunshine (hours)', 'Speed of maximum wind gust (km/h)' \
#                 ]].corr(method='pearson')
# corr = dataset[['Minimum temperature (°C)', 'Maximum temperature (°C)', 'Evaporation (mm)', \
#                   '9am Temperature (°C)', '3pm Temperature (°C)' ]].corr(method='pearson')

corr

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Speed of maximum wind gust (km/h),9am Temperature (°C),3pm Temperature (°C)
Minimum temperature (°C),1.000000,0.707525,0.043350,0.655091,0.081852,0.055604,0.915726,0.661579
Maximum temperature (°C),0.707525,1.000000,-0.124851,0.620727,0.469026,-0.056150,0.820029,0.965177
Rainfall (mm),0.043350,-0.124851,1.000000,-0.044602,-0.146490,0.041965,-0.021033,-0.126947
Evaporation (mm),0.655091,0.620727,-0.044602,1.000000,0.273256,0.158113,0.705851,0.560214
Sunshine (hours),0.081852,0.469026,-0.146490,0.273256,1.000000,-0.058398,0.197034,0.487546
Speed of maximum wind gust (km/h),0.055604,-0.056150,0.041965,0.158113,-0.058398,1.000000,0.108985,-0.097792
9am Temperature (°C),0.915726,0.820029,-0.021033,0.705851,0.197034,0.108985,1.000000,0.761603
3pm Temperature (°C),0.661579,0.965177,-0.126947,0.560214,0.487546,-0.097792,0.761603,1.000000


### 1st model: Predict max daily energy demand on weather
### LINEAR REGRESSION MODEL

In [157]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import linear_model

# Choose the features (input)
X = dataset[['Minimum temperature (°C)', 'Maximum temperature (°C)', 'Evaporation (mm)', '9am Temperature (°C)', '3pm Temperature (°C)']]

# What we have to predict (output)
Y = dataset['Max_TOTALDEMAND']

# Splitting the data into training set and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state = 1)

# Generate the regression model (import .linear_model to use .LinearRegression() func)
lm = linear_model.LinearRegression()

# Create/ Produce the model: Fit training data into model
model = lm.fit(X_train, Y_train)
display(model)

# Predict coefficient & intercept from linear regression model(lm)
print(lm.coef_, lm.intercept_)

LinearRegression()

[-179.74679919  138.97436703   -7.95064206   67.8489649  -137.16496524] 7061.4147822456125


In [166]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


X= dataset[['Minimum temperature (°C)','Maximum temperature (°C)','Evaporation (mm)','Sunshine (hours)','Speed of maximum wind gust (km/h)',\
          '9am Temperature (°C)','9am wind speed (km/h)','9am MSL pressure (hPa)','3pm Temperature (°C)',\
          '3pm wind speed (km/h)','3pm MSL pressure (hPa)']].astype(float)

y = dataset['Max_TOTALDEMAND']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# partial code here...
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
y_test_predictions = lm.predict(X_test)
print('actual TOTALDEMAND values of the first 5 test data:')
print(y_test[0:5])
print('')
print('predicted TOTALDEMAND values of the first 5 test data:')
print(y_test_predictions[0:5])
print('')

# coefficients

print('Coefficients: ', end = ' ')
print(lm.coef_)
print('')

# intercept:
print('Intercept: ', end = ' ')
print(lm.intercept_)
print('')

# R^2
r2_test = lm.score(X_test, y_test)
r2_train = lm.score(X_train, y_train)

print('Coefficient of determination (test): {0:.2f}'.format(r2_test))
print('Coefficient of determination (training): {0:.2f}'.format(r2_train))

actual TOTALDEMAND values of the first 5 test data:
162    6062.58
60     5459.22
61     5526.41
63     5319.54
69     5626.12
Name: Max_TOTALDEMAND, dtype: float64

predicted TOTALDEMAND values of the first 5 test data:
[6531.22923604 5829.73873573 5554.4334064  5339.60215168 6390.04188686]

Coefficients:  [-133.17402238  115.75211913   15.85974956  -32.47551386    6.24241556
   -7.5918158    13.28663814   53.24416282 -103.73522371  -18.98214091
  -59.56510237]

Intercept:  13635.183087106043

Coefficient of determination (test): 0.17
Coefficient of determination (training): 0.39


#### Access how regression model is doing

In [158]:
from sklearn.metrics import mean_squared_error, r2_score
r2_test = lm.score(X_test, Y_test) #on testing data
print(r2_test)

0.22770246679688189


### 2nd model: Predicts Max Daily Price: CLASSIFICATION: KNN

In [159]:
import pandas as pd
from sklearn.model_selection import train_test_split # For splitting
from sklearn.metrics import accuracy_score # To check accuracy of the prediction
from sklearn import neighbors # To produce/generate KNeighborsClassifier
from sklearn import preprocessing # To scale/normalise the features

# Select features (input)
features = dataset[['Minimum temperature (°C)', 'Maximum temperature (°C)', 'Evaporation (mm)', 'Sunshine (hours)','9am Temperature (°C)', '3pm Temperature (°C)' ]]

# What we want to predict (output)
classlabel = dataset['Max_PRICECATEGORY']

# Splitting
features_train, features_test, class_train, class_test = train_test_split(features, classlabel, train_size = 0.8, random_state = 1)

# Scale/Normalize the features
scaler = preprocessing.StandardScaler().fit(features_train) 
features_train = scaler.transform(features_train) 
features_test = scaler.transform(features_test)

# Generating KNN classifier model & import neighbors from sklearn library
knn = neighbors.KNeighborsClassifier(n_neighbors = 5)

# Creating model: Fitting features & classlabel in training data set
knn.fit(features_train, class_train)

# Produce predictions & check its .accuracy_score() on testing data set
predictions = knn.predict(features_test)
print(accuracy_score(class_test, predictions))


0.5714285714285714


c:\Users\lhong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### 2nd model: Predict max daily price: CLASSIFICATION: DECISION TREES

In [160]:
import pandas as pd
from sklearn.model_selection import train_test_split # For splitting
from sklearn.tree import DecisionTreeClassifier # For generating the model
from sklearn.metrics import accuracy_score # To check accuracy of the prediction
from sklearn import preprocessing

# Select features (input)
features = dataset[['Minimum temperature (°C)', 'Maximum temperature (°C)', 'Evaporation (mm)', 'Sunshine (hours)','9am Temperature (°C)', '3pm Temperature (°C)']]

# What we want to predict (output)
classlabel = dataset['Max_PRICECATEGORY']

# Splitting
features_train, features_test, class_train, class_test = train_test_split(features, classlabel, train_size = 0.8, random_state = 1)

# scaling/Normalizing the values
scaler = preprocessing.StandardScaler().fit(features_train)
features_train = scaler.transform(features_train)
features_test = scaler.transform(features_test)

#  Generating the decision tree model
dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 1)

# Create tree = Fitting features and classlabel into the model
dt.fit(features_train, class_train)

# Check the accuracy_score of the prediction
predictions = dt.predict(features_test) # predictions based on testing set
print(accuracy_score(class_test, predictions)) # report how well these predictiosn based on acc_score

0.3877551020408163


## Chi Squared

In [174]:
import scipy.stats as stats

data = pd.DataFrame(np.array([[1,1,1],[1,0,1],[0,1,0],[0,0,0]]), \
            columns=['Minimum temperature (°C)','Maximum temperature (°C)','Max_PRICECATEGORY'])
features=data[['Minimum temperature (°C)','Maximum temperature (°C)']]
class_label = data['Max_PRICECATEGORY']
cont_table = pd.crosstab(class_label,features['Maximum temperature (°C)'])
chi2_val, p, dof, expected = stats.chi2_contingency(cont_table.values, correction=False)
print('Chi2 value: ',chi2_val)
if(p<0.05) : 
    print('Null hypothesis rejected, p value: ', p)
else :
    print('Null hypothesis accepted, p value: ', p)

Chi2 value:  0.0
Null hypothesis accepted, p value:  1.0


In [161]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

dt = dataset[['Minimum temperature (°C)','Maximum temperature (°C)','Rainfall (mm)',\
            'Evaporation (mm)','Sunshine (hours)','Speed of maximum wind gust (km/h)',\
            '9am Temperature (°C)','9am relative humidity (%)'\
           ,'9am cloud amount (oktas)','9am wind speed (km/h)','9am MSL pressure (hPa)',\
            '3pm Temperature (°C)','3pm relative humidity (%)','3pm cloud amount (oktas)',\
            '3pm wind speed (km/h)','3pm MSL pressure (hPa)']]
cl = dataset['Max_PRICECATEGORY']

X_train, X_test, y_train, y_test = train_test_split(dt,cl, train_size =0.66, random_state = 42)


# Instantiate
feature_selector = SelectKBest(chi2, k=3)

# Perform selection
X_train = feature_selector.fit_transform(X_train, y_train)
X_test = feature_selector.transform(X_test)


#Scale the data
scaler = preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

#Impute missing values via mean imputation
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)
    
#Train k-nn classifier
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
dt = DecisionTreeClassifier(criterion="entropy", max_depth = 5)
    
# STEP 2: Fitting data / Training
knn.fit(X_train, y_train)
dt.fit(X_train, y_train)

# STEP 3: Prediction / Test
y_pred=knn.predict(X_test)
y_pred_dt = dt.predict(X_test)
    
# STEP 4: Eval
acc_score.append(accuracy_score(y_test, y_pred))
acc_score_dt.append(accuracy_score(y_test, y_pred_dt))
    
print(acc_score)
#Display average of accuracy scores
avg_acc_score = sum(acc_score)/k
print(avg_acc_score)



print (acc_score_dt)
avg_acc_score_dt = sum(acc_score_dt)/k
print(avg_acc_score_dt)

[0.4819277108433735]
0.0963855421686747
[0.46987951807228917]
0.09397590361445783


c:\Users\lhong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## PCA

In [168]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

data = dt

classlabel = dataset['Max_TOTALDEMAND']

# if too much k-fold, e.g. k=100, too much computational power 
k=5

# Initiation
kf = KFold(n_splits=k, shuffle=True, random_state=200) # change random_state = 125 and run next cell sum(...)/k

acc_score = []
acc_score_dt = []

for train_index, test_index in kf.split(data):
    #Perform the split for this fold
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index, :]
    y_train, y_test = classlabel[train_index], classlabel[test_index]
    
    
    # STEP 0
    scaler = preprocessing.StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)

    imputer = SimpleImputer()
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)
    
    # Instantiate
    # Option 1: SelectKBest method
    feature_selector = SelectKBest(mutual_info_classif, k=15)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)

    # Option 2: PCA
    pca = PCA(n_components=15)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    # STEP 1
    knn = neighbors.KNeighborsClassifier(n_neighbors=5) #  try n_neighbors 5 vs 3
    dt = DecisionTreeClassifier(criterion="entropy", max_depth=5)

    # STEP 2: Fitting data / Training
    knn.fit(X_train, y_train)
    dt.fit(X_train, y_train)

    # STEP 3: Prediction / Test
    y_pred=knn.predict(X_test)
    y_pred_dt = dt.predict(X_test)
    
    # STEP 4: Eval
    acc_score.append(accuracy_score(y_test, y_pred))
    acc_score_dt.append(accuracy_score(y_test, y_pred_dt))

TypeError: Singleton array array(DecisionTreeClassifier(criterion='entropy', max_depth=5),
      dtype=object) cannot be considered a valid collection.